In [5]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [6]:
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Trapani,IT,2021-04-12 14:42:11,37.8333,12.6667,68.00,60,0,16.11
1,1,Barrow,US,2021-04-12 14:41:35,71.2906,-156.7887,-18.40,69,1,6.91
2,2,Rio Gallegos,AR,2021-04-12 14:41:40,-51.6226,-69.2181,48.20,57,0,16.11
3,3,Bria,CF,2021-04-12 14:45:06,6.5423,21.9863,81.18,59,100,5.08
4,4,Tera,NE,2021-04-12 14:45:06,14.0078,0.7531,108.03,3,0,14.00


In [7]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)


In [8]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [9]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [12]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Bria,CF,2021-04-12 14:45:06,6.5423,21.9863,81.18,59,100,5.08
6,6,Cape Town,ZA,2021-04-12 14:43:37,-33.9258,18.4232,75.99,53,0,14.97
7,7,Hermanus,ZA,2021-04-12 14:44:02,-34.4187,19.2345,77.00,60,100,1.99
10,10,Vila Velha,BR,2021-04-12 14:45:11,-20.3297,-40.2925,86.00,54,40,8.05
11,11,Bengkulu,ID,2021-04-12 14:42:05,-3.8004,102.2655,77.22,86,96,4.00
17,17,Buluang,PH,2021-04-12 14:45:16,13.3068,123.3425,75.87,91,76,3.09
18,18,San Andres,CO,2021-04-12 14:43:21,12.5847,-81.7006,84.20,74,40,8.05
22,22,La Plaine,DM,2021-04-12 14:45:19,15.3333,-61.2500,84.20,70,75,2.30
23,23,Lorengau,PG,2021-04-12 14:45:20,-2.0226,147.2712,78.04,82,100,4.61
24,24,Miri,MY,2021-04-12 14:45:21,4.4148,114.0089,78.80,89,40,0.65


In [13]:
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Bria,CF,81.18,6.5423,21.9863,
6,Cape Town,ZA,75.99,-33.9258,18.4232,
7,Hermanus,ZA,77.00,-34.4187,19.2345,
10,Vila Velha,BR,86.00,-20.3297,-40.2925,
11,Bengkulu,ID,77.22,-3.8004,102.2655,
17,Buluang,PH,75.87,13.3068,123.3425,
18,San Andres,CO,84.20,12.5847,-81.7006,
22,La Plaine,DM,84.20,15.3333,-61.2500,
23,Lorengau,PG,78.04,-2.0226,147.2712,
24,Miri,MY,78.80,4.4148,114.0089,


In [14]:
# Set parameters to search for a hotel.
params = {
    "radius": 500,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?parameters"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [15]:
hotel_df.tail(20)

,City,Country,Max Temp,Lat,Lng,Hotel Name
502,Tomatlan,MX,77.32,19.9333,-105.2500,Norma franco
505,Kisangani,CD,87.64,0.5167,25.2000,Centre D'accueil Joli Reve
507,Isabela,PH,80.87,6.7085,121.9711,Casa Rosario
508,Lazaro Cardenas,MX,78.58,17.9583,-102.2000,Hotel Viña Del Mar
513,Sal Rei,CV,82.40,16.1833,-22.9167,Ca Jasmine
514,Miranpur,IN,86.02,29.3000,77.9333,Dhaba New Chaudhari
519,San Patricio,US,78.01,28.0170,-97.5169,
520,Ler,SS,81.30,8.3018,30.1418,
521,Ambilobe,MG,78.66,-13.2000,49.0500,Hotel National
522,Wanning,CN,75.34,18.8003,110.3967,Yelinwan Hotel


In [16]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [19]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))